In [1]:
import pandas as pd
import plotly.graph_objects as go
from evento import ajustar_datas

In [42]:
# Fundo sem histórico completo de preços no Yahoo Finance
# Carregar histórico da B3
precos = (
    pd.read_csv('data/knri.csv', parse_dates=['datneg'])
    # a série da B3 começa em "2010-12-01" -> vamos começar em "2011-01-01"
    .query('datneg >= "2011-01-01"')
)
precos

,datneg,preult
20,2011-01-04,100.00
21,2011-01-05,99.70
22,2011-01-06,99.70
23,2011-01-07,99.50
24,2011-01-10,95.50
...,...,...
2987,2022-12-29,140.20
2988,2023-01-02,139.81
2989,2023-01-03,139.00
2990,2023-01-04,136.65


In [43]:
# Histórico de dividendos do HGLG11 extraído do site Funds Explorer
divs = (
    pd.read_csv('data/knri_divs.csv', parse_dates=['data_com', 'data_pag'])
    # a série da B3 começa em "2010-12-01" -> vamos começar em "2011-01-01"
    .query('data_com >= "2011-01-01"')

)
divs

,tipo,data_com,data_pag,dividendo
1,Rendimento,2011-01-31,2011-02-14,0.675
2,Rendimento,2011-02-28,2011-03-16,0.700
3,Rendimento,2011-03-31,2011-04-14,0.700
4,Rendimento,2011-04-29,2011-05-13,0.700
5,Rendimento,2011-05-31,2011-06-14,0.700
...,...,...,...,...
140,Rendimento,2022-08-31,2022-09-15,0.910
141,Rendimento,2022-09-30,2022-10-17,0.910
142,Rendimento,2022-10-31,2022-11-16,0.910
143,Rendimento,2022-11-30,2022-12-14,0.910


In [44]:
# Fazer um gráfico de barras com o dividendos pagos pela KNRI11 a cada ano usano o plotly
# Calcular o ano de cada data-com
divs["ano"] = divs["data_com"].dt.year
# Agrupar os dividendos por ano
divs_gb = divs.groupby("ano")["dividendo"].sum().reset_index()
# Criar o gráfico
fig = go.Figure(
    data=[
        go.Bar(
            x=divs_gb["ano"],
            y=divs_gb["dividendo"],
            text=divs_gb["dividendo"].round(1),
            textposition="auto",
        )
    ]
)
fig.update_layout(
    title="Dividendos pagos pela KNRI11 a cada ano",
    xaxis_title="Ano",
    yaxis_title="Dividendos (R$)",
)
fig.show()

In [45]:
# Existe amortização de capital?
divs["tipo"].value_counts()

Rendimento    144
Name: tipo, dtype: int64

In [46]:
# As colunas "tipo" e "ano" não serão mais usadas
divs.drop(columns=["tipo", "ano"], inplace=True)

In [47]:
# Verificar a quantidade de datas a serem ajustadas
print(len(set(divs['data_com']) - set(ajustar_datas(divs['data_com'], precos['datneg']))), 'datas-com a serem ajustadas')
# Assegurar que as datas dos eventos sejam em datas com negociação do ativo
# anterior=True -> se não houver negociação na data_com, ajustar para a primeira data anterior de negociação
divs['data_com'] = ajustar_datas(divs['data_com'], precos['datneg'])
# anterior=False -> se não houver negociação na data de pagamento, ajustar para a primeira data posterior
divs['data_pag'] = ajustar_datas(divs['data_pag'], precos['datneg'], anterior=False)
divs

0 datas-com a serem ajustadas


,data_com,data_pag,dividendo
1,2011-01-31,2011-02-14,0.675
2,2011-02-28,2011-03-16,0.700
3,2011-03-31,2011-04-14,0.700
4,2011-04-29,2011-05-13,0.700
5,2011-05-31,2011-06-14,0.700
...,...,...,...
140,2022-08-31,2022-09-15,0.910
141,2022-09-30,2022-10-17,0.910
142,2022-10-31,2022-11-16,0.910
143,2022-11-30,2022-12-14,0.910


In [48]:
# Verificar se existe alguma "data_com" repetida
divs['data_com'].duplicated().any()

False

In [49]:
# Juntar o histórico de preços com o histórico de dividendos no dataframe "tr" (total return)
# A coluna "data_pag" será usada em etapa posterior
tr = pd.merge(precos, divs[["data_com", "dividendo"]], how="left", left_on="datneg", right_on="data_com")
# Sinalizar que esses dividendos são na data-com -> renomear de "dividendo" para "div_data_com"
tr.rename(columns={'dividendo': 'div_data_com'}, inplace=True)
# Data-com é a data em que existe "dividendo" -> remover coluna "data_com"
tr.drop(columns=["data_com"], inplace=True)
tr

,datneg,preult,div_data_com
0,2011-01-04,100.00,NaN
1,2011-01-05,99.70,NaN
2,2011-01-06,99.70,NaN
3,2011-01-07,99.50,NaN
4,2011-01-10,95.50,NaN
...,...,...,...
2967,2022-12-29,140.20,1.0
2968,2023-01-02,139.81,NaN
2969,2023-01-03,139.00,NaN
2970,2023-01-04,136.65,NaN


In [50]:
# Verificar o início do dataframe "tr" para ver se está tudo certo
tr.loc[15:20]

,datneg,preult,div_data_com
15,2011-01-27,97.00,NaN
16,2011-01-28,96.60,NaN
17,2011-01-31,96.99,0.675
18,2011-02-01,96.99,NaN
19,2011-02-02,99.00,NaN
20,2011-02-03,98.50,NaN


In [51]:
# Criar nova coluna "div_data_ex" que é o valor do dividendo na data-ex (ex-dividendos)
tr["div_data_ex"] = tr["div_data_com"].shift(1)
# Só a coluna "div_data_ex" será usada -> remover a coluna "div_data_com"
tr.drop(columns=["div_data_com"], inplace=True)
# Verificar o início do dataframe "tr" para ver se está tudo certo
tr.loc[15:20]

,datneg,preult,div_data_ex
15,2011-01-27,97.00,NaN
16,2011-01-28,96.60,NaN
17,2011-01-31,96.99,NaN
18,2011-02-01,96.99,0.675
19,2011-02-02,99.00,NaN
20,2011-02-03,98.50,NaN


In [52]:
# Inserir o histórico de dividendos na "data_pag" no dataframe "tr" (total return)
tr = tr.merge(divs[["data_pag", "dividendo"]], how="left", left_on="datneg", right_on="data_pag")
# Sinalizar que esses dividendos são na data-pag -> renomear de "dividendo" para "div_data_pag"
tr.rename(columns={'dividendo': 'div_data_pag'}, inplace=True)
# Data-pagm é a data em que existe "div_data_pag" -> remover coluna "data_pag"
tr.drop(columns=["data_pag"], inplace=True)
tr

,datneg,preult,div_data_ex,div_data_pag
0,2011-01-04,100.00,NaN,NaN
1,2011-01-05,99.70,NaN,NaN
2,2011-01-06,99.70,NaN,NaN
3,2011-01-07,99.50,NaN,NaN
4,2011-01-10,95.50,NaN,NaN
...,...,...,...,...
2967,2022-12-29,140.20,NaN,NaN
2968,2023-01-02,139.81,1.0,NaN
2969,2023-01-03,139.00,NaN,NaN
2970,2023-01-04,136.65,NaN,NaN


In [53]:
tr.loc[15:30]

,datneg,preult,div_data_ex,div_data_pag
15,2011-01-27,97.00,NaN,NaN
16,2011-01-28,96.60,NaN,NaN
17,2011-01-31,96.99,NaN,NaN
18,2011-02-01,96.99,0.675,NaN
19,2011-02-02,99.00,NaN,NaN
20,2011-02-03,98.50,NaN,NaN
21,2011-02-04,99.80,NaN,NaN
22,2011-02-07,97.00,NaN,NaN
23,2011-02-08,96.50,NaN,NaN
24,2011-02-09,97.00,NaN,NaN


In [54]:
# Fazer um shift de 1 pregão para o cálculo do retorno
tr["preult_ant_1p"] = tr["preult"].shift(1)
# Mover a coluna "preult_ant_1p" para a posição 2
tr.insert(2, "preult_ant_1p", tr.pop("preult_ant_1p"))
tr.loc[15:30]

,datneg,preult,preult_ant_1p,div_data_ex,div_data_pag
15,2011-01-27,97.00,97.50,NaN,NaN
16,2011-01-28,96.60,97.00,NaN,NaN
17,2011-01-31,96.99,96.60,NaN,NaN
18,2011-02-01,96.99,96.99,0.675,NaN
19,2011-02-02,99.00,96.99,NaN,NaN
20,2011-02-03,98.50,99.00,NaN,NaN
21,2011-02-04,99.80,98.50,NaN,NaN
22,2011-02-07,97.00,99.80,NaN,NaN
23,2011-02-08,96.50,97.00,NaN,NaN
24,2011-02-09,97.00,96.50,NaN,NaN


In [55]:
# Calculando os fatores de reinvestimento de dividendos
# fr_de = 1 + dividendo / (preço de fechamento na data-ex)
# fr_dc = 1 + dividendo / (preço de fechamento na data de pagamento)
# fr_yf (Yahoo Finance) = 1 + dividendo / preço de abertura teórico na data-ex
# fr_yf = 1 + dividendo / (preço de fechamento no dia anterior - dividendo)
tr["fr_de"] = 1 + tr["div_data_ex"] / tr["preult"]
tr["fr_dc"] = 1 + tr["div_data_pag"] / tr["preult"]
tr["fr_yf"] = 1 + tr["div_data_ex"] / (tr["preult_ant_1p"] - tr["div_data_ex"])
# A coluna "preult_ant_1p" não será mais usada -> remover
tr.drop(columns=["preult_ant_1p"], inplace=True)
tr

,datneg,preult,div_data_ex,div_data_pag,fr_de,fr_dc,fr_yf
0,2011-01-04,100.00,NaN,NaN,NaN,NaN,NaN
1,2011-01-05,99.70,NaN,NaN,NaN,NaN,NaN
2,2011-01-06,99.70,NaN,NaN,NaN,NaN,NaN
3,2011-01-07,99.50,NaN,NaN,NaN,NaN,NaN
4,2011-01-10,95.50,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2967,2022-12-29,140.20,NaN,NaN,NaN,NaN,NaN
2968,2023-01-02,139.81,1.0,NaN,1.007153,NaN,1.007184
2969,2023-01-03,139.00,NaN,NaN,NaN,NaN,NaN
2970,2023-01-04,136.65,NaN,NaN,NaN,NaN,NaN


In [56]:
tr.loc[15:30]

,datneg,preult,div_data_ex,div_data_pag,fr_de,fr_dc,fr_yf
15,2011-01-27,97.00,NaN,NaN,NaN,NaN,NaN
16,2011-01-28,96.60,NaN,NaN,NaN,NaN,NaN
17,2011-01-31,96.99,NaN,NaN,NaN,NaN,NaN
18,2011-02-01,96.99,0.675,NaN,1.006959,NaN,1.007008
19,2011-02-02,99.00,NaN,NaN,NaN,NaN,NaN
20,2011-02-03,98.50,NaN,NaN,NaN,NaN,NaN
21,2011-02-04,99.80,NaN,NaN,NaN,NaN,NaN
22,2011-02-07,97.00,NaN,NaN,NaN,NaN,NaN
23,2011-02-08,96.50,NaN,NaN,NaN,NaN,NaN
24,2011-02-09,97.00,NaN,NaN,NaN,NaN,NaN


In [57]:
# Preencher os valores NaN com 1 em fr_de, fr_dc e fr_yf para calcular o produtório
tr[["fr_de", "fr_dc", "fr_yf"]] = tr[["fr_de", "fr_dc", "fr_yf"]].fillna(1)
tr

,datneg,preult,div_data_ex,div_data_pag,fr_de,fr_dc,fr_yf
0,2011-01-04,100.00,NaN,NaN,1.000000,1.0,1.000000
1,2011-01-05,99.70,NaN,NaN,1.000000,1.0,1.000000
2,2011-01-06,99.70,NaN,NaN,1.000000,1.0,1.000000
3,2011-01-07,99.50,NaN,NaN,1.000000,1.0,1.000000
4,2011-01-10,95.50,NaN,NaN,1.000000,1.0,1.000000
...,...,...,...,...,...,...,...
2967,2022-12-29,140.20,NaN,NaN,1.000000,1.0,1.000000
2968,2023-01-02,139.81,1.0,NaN,1.007153,1.0,1.007184
2969,2023-01-03,139.00,NaN,NaN,1.000000,1.0,1.000000
2970,2023-01-04,136.65,NaN,NaN,1.000000,1.0,1.000000


In [58]:
# Calcular o produtório dos fatores de reinvestimento
tr["participacao_de"] = tr["fr_de"].cumprod()
tr["participacao_dp"] = tr["fr_dc"].cumprod()
tr["participacao_yf"] = tr["fr_yf"].cumprod()
# As colunas "fr_de", "fr_dc" e "fr_yf" não serão mais usadas -> remover
tr.drop(columns=["fr_de", "fr_dc", "fr_yf"], inplace=True)
tr

,datneg,preult,div_data_ex,div_data_pag,participacao_de,participacao_dp,participacao_yf
0,2011-01-04,100.00,NaN,NaN,1.000000,1.000000,1.000000
1,2011-01-05,99.70,NaN,NaN,1.000000,1.000000,1.000000
2,2011-01-06,99.70,NaN,NaN,1.000000,1.000000,1.000000
3,2011-01-07,99.50,NaN,NaN,1.000000,1.000000,1.000000
4,2011-01-10,95.50,NaN,NaN,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...
2967,2022-12-29,140.20,NaN,NaN,2.287333,2.288011,2.287572
2968,2023-01-02,139.81,1.0,NaN,2.303694,2.288011,2.304006
2969,2023-01-03,139.00,NaN,NaN,2.303694,2.288011,2.304006
2970,2023-01-04,136.65,NaN,NaN,2.303694,2.288011,2.304006


In [59]:
tr["preult_tr"] = tr["preult"] * tr["participacao_de"]
tr

,datneg,preult,div_data_ex,div_data_pag,participacao_de,participacao_dp,participacao_yf,preult_tr
0,2011-01-04,100.00,NaN,NaN,1.000000,1.000000,1.000000,100.000000
1,2011-01-05,99.70,NaN,NaN,1.000000,1.000000,1.000000,99.700000
2,2011-01-06,99.70,NaN,NaN,1.000000,1.000000,1.000000,99.700000
3,2011-01-07,99.50,NaN,NaN,1.000000,1.000000,1.000000,99.500000
4,2011-01-10,95.50,NaN,NaN,1.000000,1.000000,1.000000,95.500000
...,...,...,...,...,...,...,...,...
2967,2022-12-29,140.20,NaN,NaN,2.287333,2.288011,2.287572,320.684147
2968,2023-01-02,139.81,1.0,NaN,2.303694,2.288011,2.304006,322.079421
2969,2023-01-03,139.00,NaN,NaN,2.303694,2.288011,2.304006,320.213429
2970,2023-01-04,136.65,NaN,NaN,2.303694,2.288011,2.304006,314.799749


In [60]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=tr["datneg"],
        y=tr["preult"],
        mode="lines",
        name="cota sem ajuste",
        line=dict(color="#e86f00"),
    )
)
fig.add_trace(
    go.Scatter(
        x=tr["datneg"],
        y=tr["preult_tr"],
        mode="lines",
        name="cota ajustada (Total Return)",
        line=dict(color="#02878e"),
    )
)
fig.update_layout(
    font=dict(family="Fira Code", size=11, color="black"),
    title="KNRI11 Total Return <br> (reinvestimento dos dividendos na data-ex)",
    title_x=0.5,
    title_y=0.85,
    xaxis_title="Data",
    yaxis_title="Preço (R$)",
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
)
fig.show()

In [61]:
# http://estatisticas.cetip.com.br/astec/series_v05/paginas/lum_web_v05_template_informacoes_di.asp?str_Modulo=completo&int_Idioma=1&int_Titulo=6&int_NivelBD=2
di = pd.read_csv("data/taxa_di.csv", sep=",", decimal=".", parse_dates=["data"], dayfirst=True)
di

,data,media,fator_diario,taxa_selic
0,2011-01-04,10.64,1.000401,10.66
1,2011-01-05,10.64,1.000401,10.67
2,2011-01-06,10.64,1.000401,10.66
3,2011-01-07,10.64,1.000401,10.67
4,2011-01-10,10.64,1.000401,10.66
...,...,...,...,...
3012,2022-12-30,13.65,1.000508,13.65
3013,2023-01-02,13.65,1.000508,13.65
3014,2023-01-03,13.65,1.000508,13.65
3015,2023-01-04,13.65,1.000508,13.65


In [62]:
di["di_acum"] = di["fator_diario"].cumprod()
di

,data,media,fator_diario,taxa_selic,di_acum
0,2011-01-04,10.64,1.000401,10.66,1.000401
1,2011-01-05,10.64,1.000401,10.67,1.000803
2,2011-01-06,10.64,1.000401,10.66,1.001204
3,2011-01-07,10.64,1.000401,10.67,1.001606
4,2011-01-10,10.64,1.000401,10.66,1.002008
...,...,...,...,...,...
3012,2022-12-30,13.65,1.000508,13.65,2.794351
3013,2023-01-02,13.65,1.000508,13.65,2.795770
3014,2023-01-03,13.65,1.000508,13.65,2.797190
3015,2023-01-04,13.65,1.000508,13.65,2.798610


In [63]:
di.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3017 entries, 0 to 3016
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   data          3017 non-null   datetime64[ns]
 1   media         3017 non-null   float64       
 2   fator_diario  3017 non-null   float64       
 3   taxa_selic    3017 non-null   float64       
 4   di_acum       3017 non-null   float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 118.0 KB


In [64]:
tr = tr.merge(di[["data", "di_acum"]], how="left", left_on="datneg", right_on="data")
tr.drop(columns=["data"], inplace=True)
tr

,datneg,preult,div_data_ex,div_data_pag,participacao_de,participacao_dp,participacao_yf,preult_tr,di_acum
0,2011-01-04,100.00,NaN,NaN,1.000000,1.000000,1.000000,100.000000,1.000401
1,2011-01-05,99.70,NaN,NaN,1.000000,1.000000,1.000000,99.700000,1.000803
2,2011-01-06,99.70,NaN,NaN,1.000000,1.000000,1.000000,99.700000,1.001204
3,2011-01-07,99.50,NaN,NaN,1.000000,1.000000,1.000000,99.500000,1.001606
4,2011-01-10,95.50,NaN,NaN,1.000000,1.000000,1.000000,95.500000,1.002008
...,...,...,...,...,...,...,...,...,...
2967,2022-12-29,140.20,NaN,NaN,2.287333,2.288011,2.287572,320.684147,2.792932
2968,2023-01-02,139.81,1.0,NaN,2.303694,2.288011,2.304006,322.079421,2.795770
2969,2023-01-03,139.00,NaN,NaN,2.303694,2.288011,2.304006,320.213429,2.797190
2970,2023-01-04,136.65,NaN,NaN,2.303694,2.288011,2.304006,314.799749,2.798610


In [65]:
tr["di_acum"] = tr["preult"].iloc[0] * tr["di_acum"]
tr

,datneg,preult,div_data_ex,div_data_pag,participacao_de,participacao_dp,participacao_yf,preult_tr,di_acum
0,2011-01-04,100.00,NaN,NaN,1.000000,1.000000,1.000000,100.000000,100.040132
1,2011-01-05,99.70,NaN,NaN,1.000000,1.000000,1.000000,99.700000,100.080280
2,2011-01-06,99.70,NaN,NaN,1.000000,1.000000,1.000000,99.700000,100.120444
3,2011-01-07,99.50,NaN,NaN,1.000000,1.000000,1.000000,99.500000,100.160625
4,2011-01-10,95.50,NaN,NaN,1.000000,1.000000,1.000000,95.500000,100.200821
...,...,...,...,...,...,...,...,...,...
2967,2022-12-29,140.20,NaN,NaN,2.287333,2.288011,2.287572,320.684147,279.293224
2968,2023-01-02,139.81,1.0,NaN,2.303694,2.288011,2.304006,322.079421,279.576991
2969,2023-01-03,139.00,NaN,NaN,2.303694,2.288011,2.304006,320.213429,279.718983
2970,2023-01-04,136.65,NaN,NaN,2.303694,2.288011,2.304006,314.799749,279.861047


In [66]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=tr["datneg"],
        y=tr["preult_tr"],
        mode="lines",
        name="Cota Ajustada (Total Return)",
        line=dict(color="#e86f00"),
    )
)
fig.add_trace(
    go.Scatter(
        x=tr["datneg"],
        y=tr["di_acum"],
        mode="lines",
        name="DI Acumulado",
        line=dict(color="#02878e"),
    )
)
fig.update_layout(
    font=dict(family="Fira Code", size=11, color="black"),
    title="KNRI11 Total Return <br> (reinvestimento dos dividendos na data-ex)",
    title_x=0.5,
    title_y=0.85,
    xaxis_title="Data",
    yaxis_title="Preço (R$)",
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
)
fig.show()

In [67]:
ipca = (pd
    .read_csv('data/ipca.csv', parse_dates=['mes'])
    .query('mes >= "2011-01-01"')
    .rename(columns={'mes':'datneg'})
    .reset_index(drop=True)
)
ipca

,datneg,indice
0,2011-01-01,3222.42
1,2011-02-01,3248.20
2,2011-03-01,3273.86
3,2011-04-01,3299.07
4,2011-05-01,3314.58
...,...,...
138,2022-07-01,6411.95
139,2022-08-01,6388.87
140,2022-09-01,6370.34
141,2022-10-01,6407.93


In [68]:
ipca.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143 entries, 0 to 142
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   datneg  143 non-null    datetime64[ns]
 1   indice  143 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 2.4 KB


In [69]:
# Normalizar o início do índice para 100
ipca['indice_norm'] = ipca['indice'] / ipca['indice'].iloc[0]
# Coluna indice não é mais necessária
ipca.drop(columns='indice', inplace=True)
ipca

,datneg,indice_norm
0,2011-01-01,1.000000
1,2011-02-01,1.008000
2,2011-03-01,1.015963
3,2011-04-01,1.023786
4,2011-05-01,1.028600
...,...,...
138,2022-07-01,1.989793
139,2022-08-01,1.982631
140,2022-09-01,1.976881
141,2022-10-01,1.988546


In [70]:
tr = tr.merge(ipca, how='outer')
tr

,datneg,preult,div_data_ex,div_data_pag,participacao_de,participacao_dp,participacao_yf,preult_tr,di_acum,indice_norm
0,2011-01-04,100.0,NaN,NaN,1.0,1.0,1.0,100.0,100.040132,NaN
1,2011-01-05,99.7,NaN,NaN,1.0,1.0,1.0,99.7,100.080280,NaN
2,2011-01-06,99.7,NaN,NaN,1.0,1.0,1.0,99.7,100.120444,NaN
3,2011-01-07,99.5,NaN,NaN,1.0,1.0,1.0,99.5,100.160625,NaN
4,2011-01-10,95.5,NaN,NaN,1.0,1.0,1.0,95.5,100.200821,NaN
...,...,...,...,...,...,...,...,...,...,...
3025,2021-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.823490
3026,2022-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.909462
3027,2022-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.959996
3028,2022-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.990082


In [71]:
tr.sort_values('datneg', ignore_index=True, inplace=True)
tr

,datneg,preult,div_data_ex,div_data_pag,participacao_de,participacao_dp,participacao_yf,preult_tr,di_acum,indice_norm
0,2011-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,2011-01-04,100.00,NaN,NaN,1.000000,1.000000,1.000000,100.000000,100.040132,NaN
2,2011-01-05,99.70,NaN,NaN,1.000000,1.000000,1.000000,99.700000,100.080280,NaN
3,2011-01-06,99.70,NaN,NaN,1.000000,1.000000,1.000000,99.700000,100.120444,NaN
4,2011-01-07,99.50,NaN,NaN,1.000000,1.000000,1.000000,99.500000,100.160625,NaN
...,...,...,...,...,...,...,...,...,...,...
3025,2022-12-29,140.20,NaN,NaN,2.287333,2.288011,2.287572,320.684147,279.293224,NaN
3026,2023-01-02,139.81,1.0,NaN,2.303694,2.288011,2.304006,322.079421,279.576991,NaN
3027,2023-01-03,139.00,NaN,NaN,2.303694,2.288011,2.304006,320.213429,279.718983,NaN
3028,2023-01-04,136.65,NaN,NaN,2.303694,2.288011,2.304006,314.799749,279.861047,NaN


In [72]:
tr['indice_norm'].fillna(method='ffill', inplace=True)
tr

,datneg,preult,div_data_ex,div_data_pag,participacao_de,participacao_dp,participacao_yf,preult_tr,di_acum,indice_norm
0,2011-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
1,2011-01-04,100.00,NaN,NaN,1.000000,1.000000,1.000000,100.000000,100.040132,1.000000
2,2011-01-05,99.70,NaN,NaN,1.000000,1.000000,1.000000,99.700000,100.080280,1.000000
3,2011-01-06,99.70,NaN,NaN,1.000000,1.000000,1.000000,99.700000,100.120444,1.000000
4,2011-01-07,99.50,NaN,NaN,1.000000,1.000000,1.000000,99.500000,100.160625,1.000000
...,...,...,...,...,...,...,...,...,...,...
3025,2022-12-29,140.20,NaN,NaN,2.287333,2.288011,2.287572,320.684147,279.293224,1.996698
3026,2023-01-02,139.81,1.0,NaN,2.303694,2.288011,2.304006,322.079421,279.576991,1.996698
3027,2023-01-03,139.00,NaN,NaN,2.303694,2.288011,2.304006,320.213429,279.718983,1.996698
3028,2023-01-04,136.65,NaN,NaN,2.303694,2.288011,2.304006,314.799749,279.861047,1.996698


In [73]:
tr.dropna(subset=['preult'], inplace=True)
tr

,datneg,preult,div_data_ex,div_data_pag,participacao_de,participacao_dp,participacao_yf,preult_tr,di_acum,indice_norm
1,2011-01-04,100.00,NaN,NaN,1.000000,1.000000,1.000000,100.000000,100.040132,1.000000
2,2011-01-05,99.70,NaN,NaN,1.000000,1.000000,1.000000,99.700000,100.080280,1.000000
3,2011-01-06,99.70,NaN,NaN,1.000000,1.000000,1.000000,99.700000,100.120444,1.000000
4,2011-01-07,99.50,NaN,NaN,1.000000,1.000000,1.000000,99.500000,100.160625,1.000000
5,2011-01-10,95.50,NaN,NaN,1.000000,1.000000,1.000000,95.500000,100.200821,1.000000
...,...,...,...,...,...,...,...,...,...,...
3025,2022-12-29,140.20,NaN,NaN,2.287333,2.288011,2.287572,320.684147,279.293224,1.996698
3026,2023-01-02,139.81,1.0,NaN,2.303694,2.288011,2.304006,322.079421,279.576991,1.996698
3027,2023-01-03,139.00,NaN,NaN,2.303694,2.288011,2.304006,320.213429,279.718983,1.996698
3028,2023-01-04,136.65,NaN,NaN,2.303694,2.288011,2.304006,314.799749,279.861047,1.996698


In [74]:
tr['preult_tr_real'] = tr['preult_tr'] / tr['indice_norm']
tr['di_acum_real'] = tr['di_acum'] / tr['indice_norm']
tr

,datneg,preult,div_data_ex,div_data_pag,participacao_de,participacao_dp,participacao_yf,preult_tr,di_acum,indice_norm,preult_tr_real,di_acum_real
1,2011-01-04,100.00,NaN,NaN,1.000000,1.000000,1.000000,100.000000,100.040132,1.000000,100.000000,100.040132
2,2011-01-05,99.70,NaN,NaN,1.000000,1.000000,1.000000,99.700000,100.080280,1.000000,99.700000,100.080280
3,2011-01-06,99.70,NaN,NaN,1.000000,1.000000,1.000000,99.700000,100.120444,1.000000,99.700000,100.120444
4,2011-01-07,99.50,NaN,NaN,1.000000,1.000000,1.000000,99.500000,100.160625,1.000000,99.500000,100.160625
5,2011-01-10,95.50,NaN,NaN,1.000000,1.000000,1.000000,95.500000,100.200821,1.000000,95.500000,100.200821
...,...,...,...,...,...,...,...,...,...,...,...,...
3025,2022-12-29,140.20,NaN,NaN,2.287333,2.288011,2.287572,320.684147,279.293224,1.996698,160.607226,139.877541
3026,2023-01-02,139.81,1.0,NaN,2.303694,2.288011,2.304006,322.079421,279.576991,1.996698,161.306016,140.019659
3027,2023-01-03,139.00,NaN,NaN,2.303694,2.288011,2.304006,320.213429,279.718983,1.996698,160.371477,140.090772
3028,2023-01-04,136.65,NaN,NaN,2.303694,2.288011,2.304006,314.799749,279.861047,1.996698,157.660161,140.161921


In [75]:
# Plotar o gráfico
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=tr['datneg'],
        y=tr['preult_tr_real'],
        mode='lines',
        name='Cota Total Return Deflacionada pelo IPCA',
        line=dict(color='#e86f00'),
    )
)
fig.add_trace(
    go.Scatter(
        x=tr['datneg'],
        y=tr['di_acum_real'],
        mode='lines',
        name='DI Acumulado Deflacionado pelo IPCA',
        line=dict(color='#02878e'),
    )
)
fig.update_layout(
    font=dict(family='Fira Code', size=11, color='black'),
    title='KNRI11 Total Real Return <br> (reinvestimento dos dividendos e deflação pelo IPCA)',
    title_x=0.5,
    title_y=0.85,
    xaxis_title='Data',
    yaxis_title='Preço (R$)',
    legend=dict(yanchor='top', y=0.99, xanchor='left', x=0.01),
)
fig.show()


In [76]:
# Ganho real do fundo
ganho_real_periodo = (tr['preult_tr_real'].iloc[-1] - tr['preult_tr_real'].iloc[0]) / tr['preult_tr_real'].iloc[0]
print(f'Ganhor real no período  = {ganho_real_periodo:.2%}')
years = (tr['datneg'].iloc[-1] - tr['datneg'].iloc[-0]).days / 365
print(f'Total de anos no periodo = {years:.2f} anos')
# Ganho real anualizado
ganho_real_periodo_anualizado = (ganho_real_periodo + 1) ** (1 / years) - 1
print(f'Ganho real anualizado = {ganho_real_periodo_anualizado:.2%}')

Ganhor real no período  = 58.43%
Total de anos no periodo = 12.01 anos
Ganho real anualizado = 3.91%


In [77]:
# https://data.anbima.com.br/indices/?utm_source=anbima.com.br/pt_br&utm_medium=banner_indices&utm_campaign=banner_indices_portal&_ga=2.43112549.1151568226.1673173175-2062883459.1673173174
imab5 = (pd
    .read_csv('data/imab5.csv', parse_dates=['data'], dayfirst=True)
    .query('data >= "2011-01-01"')
    .rename(columns={'data':'datneg'})
    .reset_index(drop=True)
)
imab5

,datneg,imab5
0,2011-01-03,2.230022
1,2011-01-04,2.230661
2,2011-01-05,2.231823
3,2011-01-06,2.232618
4,2011-01-07,2.231839
...,...,...
3013,2023-01-02,7.982510
3014,2023-01-03,7.972746
3015,2023-01-04,7.965974
3016,2023-01-05,7.979865


In [78]:
# Normalizar o início do índice para 100
imab5['imab5_norm'] = 100* imab5['imab5'] / imab5['imab5'].iloc[0]
# Coluna indice não é mais necessária
imab5.drop(columns='imab5', inplace=True)
imab5

,datneg,imab5_norm
0,2011-01-03,100.000000
1,2011-01-04,100.028639
2,2011-01-05,100.080761
3,2011-01-06,100.116378
4,2011-01-07,100.081453
...,...,...
3013,2023-01-02,357.956514
3014,2023-01-03,357.518661
3015,2023-01-04,357.214964
3016,2023-01-05,357.837891


In [79]:
# Juntar os dois índices
tr = tr.merge(imab5, how='left')
tr

,datneg,preult,div_data_ex,div_data_pag,participacao_de,participacao_dp,participacao_yf,preult_tr,di_acum,indice_norm,preult_tr_real,di_acum_real,imab5_norm
0,2011-01-04,100.00,NaN,NaN,1.000000,1.000000,1.000000,100.000000,100.040132,1.000000,100.000000,100.040132,100.028639
1,2011-01-05,99.70,NaN,NaN,1.000000,1.000000,1.000000,99.700000,100.080280,1.000000,99.700000,100.080280,100.080761
2,2011-01-06,99.70,NaN,NaN,1.000000,1.000000,1.000000,99.700000,100.120444,1.000000,99.700000,100.120444,100.116378
3,2011-01-07,99.50,NaN,NaN,1.000000,1.000000,1.000000,99.500000,100.160625,1.000000,99.500000,100.160625,100.081453
4,2011-01-10,95.50,NaN,NaN,1.000000,1.000000,1.000000,95.500000,100.200821,1.000000,95.500000,100.200821,100.019776
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2968,2022-12-29,140.20,NaN,NaN,2.287333,2.288011,2.287572,320.684147,279.293224,1.996698,160.607226,139.877541,358.854554
2969,2023-01-02,139.81,1.0,NaN,2.303694,2.288011,2.304006,322.079421,279.576991,1.996698,161.306016,140.019659,357.956514
2970,2023-01-03,139.00,NaN,NaN,2.303694,2.288011,2.304006,320.213429,279.718983,1.996698,160.371477,140.090772,357.518661
2971,2023-01-04,136.65,NaN,NaN,2.303694,2.288011,2.304006,314.799749,279.861047,1.996698,157.660161,140.161921,357.214964


In [80]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=tr["datneg"],
        y=tr["preult_tr"],
        mode="lines",
        name="Cota Ajustada (Total Return)",
        line=dict(color="#e86f00"),
    )
)
fig.add_trace(
    go.Scatter(
        x=tr["datneg"],
        y=tr["di_acum"],
        mode="lines",
        name="DI Acumulado",
        line=dict(color="#02878e"),
    )
)
fig.add_trace(
    go.Scatter(
        x=tr["datneg"],
        y=tr["imab5_norm"],
        mode="lines",
        name="DI Acumulado",
        line=dict(color="#02878e"),
    )
)

fig.update_layout(
    font=dict(family="Fira Code", size=11, color="black"),
    title="KNRI11 Total Return <br> (reinvestimento dos dividendos na data-ex)",
    title_x=0.5,
    title_y=0.85,
    xaxis_title="Data",
    yaxis_title="Preço (R$)",
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
)
fig.show()